In [9]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(),override=True)

api_key=os.getenv("ONEAPI_API_KEY", "")
base_url=os.getenv("ONEAPI_BASE_URL", "")

bing_search_url=os.getenv("BING_SEARCH_URL", "")
bing_subscription_key=os.getenv("BING_SEARCH_API_KEY", "")

In [8]:
import json
from langchain.utilities import BingSearchAPIWrapper
from langchain_community.tools.bing_search import BingSearchResults
from pydantic import BaseModel, Field

class BingSearchAPI(BaseModel):
    bing_search_url: str = Field(default=bing_search_url)
    bing_subscription_key: str = Field(default=bing_subscription_key)
    k: int = Field(default=3)

    def search(self, question: str):
        api_wrapper = BingSearchAPIWrapper(
            bing_search_url=self.bing_search_url,
            bing_subscription_key=self.bing_subscription_key,
            k=self.k,
        )
        tool = BingSearchResults(api_wrapper=api_wrapper)
        return tool.invoke(question)

question = "什么是量子计算"

response = BingSearchAPI().search(question)
response = json.loads(response.replace("'", '"'))
for item in response:
    print(item)

{'snippet': '量子计算是一<b>种遵循量子力学规律调控量子信息单元进行计算的新型计算模式</b>。<b>与</b>经典计算不同，量子计算遵循量子力学规律，它是能突破经典算力瓶颈的新型计算模式。量子计算机，作为执行量子计算任务的设备，以量子比特（qubit）为基本运算单元。', 'title': '<b>量子计算</b>（遵循<b>量子</b>力学规律的<b>计算</b>模式）_百度百科', 'link': 'https://baike.baidu.com/item/%E9%87%8F%E5%AD%90%E8%AE%A1%E7%AE%97/11035661'}
{'snippet': '本文介绍了量子计算的基本概念、类型、优势和应用领域，以及全球量子计算产业的发展现状和前景。量子计算是利用量子力学的基本原理来加速解决复杂计算的过程，可以在各个行业提高效率和创新能力，但也面临着技术和市场的挑战。', 'title': '一文看懂<b>量子计算</b>：原理、应用、行业现状-36氪', 'link': 'https://www.36kr.com/p/1723178598401'}
{'snippet': '量子计算是利用亚原子粒子的物理学原理来执行并行计算的系统，可以在某些数学运算中更快速有效地模拟离奇的量子现象。本文介绍了量子计算的概念、作用、工作原理和现状，以及 NVIDIA cuQuantum SDK 的应用。', 'title': '科普 | <b>什么是量子计算</b>？ - 知乎', 'link': 'https://zhuanlan.zhihu.com/p/366657266'}
{'snippet': '<b>量子计算利用专门技术</b>（包括计算机硬件和利用量子力学的算法）来解决传统计算机或超级计算机无法解决或无法快速解决的复杂问题。 如今，IBM Quantum 为数十万开发人员提供了真正的量子硬件，这是一种科学家在三十年前才开始想象的工具。', 'title': '<b>什么是量子计算</b>？| IBM', 'link': 'https://www.ibm.com/cn-zh/topics/quantum-computing'}


In [12]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from typing import Any

class CustomSearchAgent(BaseModel):
    llm: Any
    search_client: Any
    question: str = None
    query_reformulation_template: str = """
        作为一个搜索专家，你需要将用户的问题转换为有效的搜索关键词。
        
        用户问题: {question}
        
        请生成2-3个最优的搜索查询词组。要求：
        1. 提取核心概念和关键实体
        2. 移除不必要的词语
        3. 考虑同义词和相关术语
        4. 确保查询词具有足够的特异性
        
        仅输出搜索词，每行一个，不要其他解释。
        """
    query_prompt: PromptTemplate = PromptTemplate(
        input_variables=["question"],
        template=query_reformulation_template
    )

    def reformulate_query(self, user_question):
        """将用户问题转换为搜索关键词"""
        # 使用LLM生成搜索词
        search_queries = self.llm(self.query_prompt.format(question=user_question))
        # 分割成列表
        return [q.strip() for q in search_queries.split('\n') if q.strip()]

    def search(self, user_question):
        """执行搜索流程"""
        # 1. 重构查询
        search_queries = self.reformulate_query(user_question)
        
        # 2. 执行多个搜索并合并结果
        all_results = []
        for query in search_queries:
            results = self.search_client.search(query, count=5)
            all_results.extend(results)
            
        # 3. 结果去重和排序
        unique_results = self._deduplicate_results(all_results)
        
        return unique_results

    def _deduplicate_results(self, results):
        """去除重复的搜索结果"""
        seen_urls = set()
        unique_results = []
        
        for result in results:
            if result['url'] not in seen_urls:
                seen_urls.add(result['url'])
                unique_results.append(result)
                
        return unique_results

In [13]:
question = "Python中如何实现多线程编程？"
# 初始化agent
agent = CustomSearchAgent(
    llm=OpenAI(api_key=api_key, base_url=base_url),
    search_client=BingSearchAPI(k=3),
    question=question
)

# 测试查询
results = agent.search(question)

# 打印结果
for result in results:
    print(f"Title: {result['title']}")
    print(f"URL: {result['url']}")
    print(f"Snippet: {result['snippet']}")
    print("---")

/tmp/ipykernel_195801/2179686752.py:30: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  search_queries = self.llm(self.query_prompt.format(question=user_question))


TypeError: BingSearchAPI.search() got an unexpected keyword argument 'count'